# Unit Simulation Experimental Notebook

In [1]:
from IPython.display import HTML
import numpy as np
import pandas as pd
from matplotlib import animation
import matplotlib.pyplot as plt
%matplotlib inline
# our simulator
import battlesim as bsm

### Trial 1. 50 Clone Troopers vs. 80 B1 battledroids

Using Random AI selection for targets.

In [2]:
trial1 = bsm.Battle("datasets/starwars-clonewars.csv")
# define 100 battledroids, 50 clones
trial1.create_army([("B1 battledroid", 80), ("Clone Trooper", 20), ("Clone Trooper", 20)])
# battle droids near (0, 0), clones near (10, 10)
trial1.apply_position([bsm.Distribution('beta',a=.5, b=.5,scale=5),
                       bsm.Distribution(loc=10), 
                       bsm.Distribution(loc=5).setx(loc=7)])
trial1.set_initial_ai(["nearest"]*3)
trial1.set_rolling_ai(["nearest","close_weak","nearest"])
# run a battle
F = trial1.simulate(max_step=1000)

In [3]:
trial1.sim_jupyter()

## Unit Quantification

How do we qualify which unit is the best?

### Approach 1: Ranking

We rank **six features**: [Range, Damage, Movement, Miss rate, HP and Accuracy], normalize, and take the mean average.

In [4]:
bsm.unit_quant.rank_score(trial1.db_)

Name
local militia               0.198413
b1 battledroid              0.244589
clone trooper               0.480880
b2 battledroid              0.482323
battle droid assassin       0.500000
clone sharpshooter          0.537879
magmaguard                  0.629509
t-series tactical droid     0.677850
bx-series droid commando    0.748918
arc trooper                 0.770924
clone commando              0.776335
dtype: float64

## Testing Performance Profiling

In [6]:
%timeit trial1.simulate()

107 ms ± 3.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
